In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt
import Computations_debug as comp
import random

#import Predict_Movie as pm
import Predict_Movie_v1 as pm
#import Predict_Movie_v2 as pm
#import Predict_Movie_v3 as pm
#import Predict_Movie_v4 as pm
pd.set_option('display.max_colwidth', 0)

In [2]:
# if you run this in colab *** it has a very limited RAM 12GB if I'm not mistaken, so works for 20K not 40K
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
predict_amazon = pm.Amazon()
#predict_amazon.debug_mode = 'On'
#predict_amazon.file_type = 'pickle'
# **** we need to filter out the movies the person has already reviewed previously ****
# because the same movie can appear in the test data-set since some new users might review it this time

In [4]:
file_name = 'data/movies.txt'; n_movies = 50000; n_movies_v = 20000; n_test_users = 100; walk_steps = 40
start_index_v = n_movies; beta = 0.1; top_neighbor = 50; top_N_movie_suggestions = 10

#file_name = 'data/movies.txt'; n_movies = 2000; n_movies_v = 2000; n_test_users = 40; walk_steps = 30
#start_index_v = n_movies; beta = 0.15; top_neighbor = 50

#lst_n_movies_test = [20000, 30000, 40000, 50000]
#lst_n_movies_test_v = [10000, 11000, 12000, 13000, 14000]
#lst_n_movies_test_v = [2000, 4000, 6000, 8000]
lst_n_movies_test_v = [20000]

start_time_ALL = datetime.datetime.now()
#for n_movies in lst_n_movies_test:
for n_movies_v in lst_n_movies_test_v:
    start_index_v = n_movies
    start_time_part1 = datetime.datetime.now()

    print('\nn_movies = {} .. n_movies_v = {} .. n_test_users = {}'.format(n_movies, n_movies_v, n_test_users))
    print('walk_steps = {} .. top_neighbor = {}'.format(walk_steps, top_neighbor))
    print('----------------------------------------------')
    print('Calculation starts for above values: {}'.format(start_time_part1))

    max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL = \
    predict_amazon.create_graphs(file_name, n_movies, n_movies_v, start_index_v)

    end_time_part1 = datetime.datetime.now()
    total_time_part1 = end_time_part1 - start_time_part1
    print('**********************************************')
    print('Calculation time for ALL-Part-1: {}'.format(total_time_part1))
    
    start_time_part2 = datetime.datetime.now()

    '''
    df_FINAL = \
    predict_amazon.run_validation(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor)
    '''
    '''
    df_FINAL = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)
    
    '''
    
    df_FINAL, df_FINAL_all = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)

    end_time_part2 = datetime.datetime.now()
    total_time_part2 = end_time_part2 - start_time_part2
    
    #start_time_part3 = datetime.datetime.now()
    # df_perf is saved in a csv, too, but it i done in the function "save_performance_values"
    df_perf = predict_amazon.save_performance_values(df_FINAL, total_time_part1, total_time_part2)
    
    df_perf_all = predict_amazon.save_performance_values_ALL(df_FINAL_all, total_time_part1, total_time_part2)
    #end_time_part3 = datetime.datetime.now()
    #total_time_part3 = end_time_part3 - start_time_part3

    # just to avoid failure if there is no RESULTS directory
    try:
        file_prefix = "RESULTS/PredictionDetails_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
        
        file_prefix_all = "RESULTS/PredictionDetails_ALL_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
    except:
        file_prefix = "PredictionDetails_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
        
        file_prefix_all = "PredictionDetails_ALL_" + str(n_movies) + "_" + str(n_movies_v) + "_" + \
        str(n_test_users) + "_" + str(walk_steps) + "_" + str(top_neighbor) + \
        "_TOP" + str(top_N_movie_suggestions) + "_"
                                                
    #fileName_Details = prs.FileNameUnique(prefix = "RESULTS/PredictionDetails_", suffix = '.csv')
    fileName_Details = prs.FileNameUnique(prefix = file_prefix, suffix = '.csv')
    fileName_Details_all = prs.FileNameUnique(prefix = file_prefix_all, suffix = '.csv')
    
    df_FINAL.to_csv(fileName_Details)
    df_FINAL_all.to_csv(fileName_Details_all)
    print('Prediction Details are saved in {}'.format(fileName_Details))
    print('Prediction_all Details are saved in {}'.format(fileName_Details_all))
    df_perf

end_time_ALL = datetime.datetime.now()
print('Calculation time for ALL-Predictions: {}'.format(end_time_ALL))

# it is taking SOOOOOO LONG time, had to stop it (even for 12K edges). Let's try in smaller networks later..
#d_model = nx.diameter(max_connected_gr_amazon_movies)
#d_test = nx.diameter(max_connected_gr_amazon_movies)

#print('Diameter of MODEL: {}\tTEST: {}'.format(d_model, d_test))

'''
>>>> df_FINAL (PredictionDetails.csv file)
> prediction: our prediction whether or not the given user will review the given movie on the list or not 
depending on the given threshold
> reality: 
> threshold: if ngbr_ratio > threshold then prediction=1 else 0
> ngbr_ratio: how many percent of your top-K similar users reviewed this movie
> tot_rev_m: how many times was this movie reviewed 
> n_old_edge: how many times did this user make a review in the old dataset
> n_new_edge: how many times did this user make a review in the new dataset
'''


n_movies = 50000 .. n_movies_v = 20000 .. n_test_users = 100
walk_steps = 40 .. top_neighbor = 50
----------------------------------------------
Calculation starts for above values: 2020-05-24 12:26:08.597514
*************** 1st Graph calculations - MODEL *************** 
Calculation time-Crete graph from the movies file: 0:00:56.345058
Is connected: True ... Is bipartite: True
# of nodes: 35940 ... # of edges: 47984
Calculation time-1st Part: 0:00:58.100757

*************** 2nd Graph calculations - TEST *************** 
Create_Bipartite_VALIDATION is running...
Calculation time-Crete graph from the movies file: 0:00:08.066555
Is connected: True ... Is bipartite: True
# of nodes: 15459 ... # of edges: 18799
n_movies for modeling (P matrix) = 1019 .. n_movies for testing = 355
n_users for modeling (P matrix) = 34921 .. n_users for testing = 15104
Avg degree (edge) per user in MODELING GRAPH: 1.3740729074196043  ... in TEST GRAPH: 1.24463718220339
Calculation time-2nd Part: 0:00:09.1740

'\n>>>> df_FINAL (PredictionDetails.csv file)\n> prediction: our prediction whether or not the given user will review the given movie on the list or not \ndepending on the given threshold\n> reality: \n> threshold: if ngbr_ratio > threshold then prediction=1 else 0\n> ngbr_ratio: how many percent of your top-K similar users reviewed this movie\n> tot_rev_m: how many times was this movie reviewed \n> n_old_edge: how many times did this user make a review in the old dataset\n> n_new_edge: how many times did this user make a review in the new dataset\n'

In [5]:
params_df = predict_amazon.show_parameters_in_use()
params_df.style.set_properties(**{'text-align': 'left'})

df_perf

,AUC,TPR,Precision,FPR,Thresholds,Calc_Time,lst_name,lst_value,other_metrics,metrics_val
0,0.801818,0.000000,0.000000,0,101,Graph Creation,debug_mode,Off,threshold,38
1,,0.316456,0.000000,0,100,0:01:07.277186,n_movies_model,50000,acc,0.948
2,,0.354430,1.000000,0,38,Random Walk & Prediction,n_movies_validation,20000,tpr,0.341772
3,,0.367089,1.000000,0.00434311,32,0:57:32.783368,walk_steps,40,fpr,0
4,,0.379747,0.878788,0.00977199,30,Threshold search,beta,0.1,tnr,1
5,,0.405063,0.769231,0.0119435,28,0:00:06.426724,top_neighbor,50,fnr,0.658228
6,,0.443038,0.744186,0.0162866,26,TOTAL TIME,n_test_users,100,precision,1
7,,0.481013,0.700000,0.0390879,24,0:58:46.487278,threshold,38,f1,0.509434
8,,0.531646,0.513514,0.0542888,22,,start_index_v,50000,,
9,,0.556962,0.456522,0.0857763,20,,file_name,data/movies.txt,,


In [6]:
df_perf_all

,AUC,TPR,Precision,FPR,Thresholds,Calc_Time,lst_name,lst_value,other_metrics,metrics_val
0,0.820591,0.000000,0.000000,0.000000,101.0,Graph Creation,debug_mode,Off,threshold,38
1,,0.139665,0.000000,0.000000,100.0,0:01:07.277186,n_movies_model,50000,acc,0.995718
2,,0.156425,1.000000,0.000000,38.0,Random Walk & Prediction,n_movies_validation,20000,tpr,0.150838
3,,0.162011,1.000000,0.000113,32.0,0:57:32.783368,walk_steps,40,fpr,0
4,,0.167598,0.878788,0.000255,30.0,Threshold search,beta,0.1,tnr,1
5,,0.178771,0.769231,0.000311,28.0,0:11:09.049255,top_neighbor,50,fnr,0.849162
6,,0.195531,0.744186,0.000425,26.0,TOTAL TIME,n_test_users,100,precision,1
7,,0.212291,0.700000,0.001019,24.0,1:09:49.109809,threshold,38,f1,0.262136
8,,0.234637,0.513514,0.001416,22.0,,start_index_v,50000,,
9,,0.245810,0.456522,0.002237,20.0,,file_name,data/movies.txt,,


In [7]:
# it is taking SOOOOOO LONG time
#d_model = nx.diameter(max_connected_gr_amazon_movies)
#d_test = nx.diameter(max_connected_gr_amazon_movies_VAL)
#print('Diameter of MODEL: {}\tTEST: {}'.format(d_model, d_test))

# centrality is fast
#centrality = nx.eigenvector_centrality_numpy(max_connected_gr_amazon_movies_VAL)
#print(centrality)

# if we use a small k, it doesn't take much time
#bc = nx.betweenness_centrality(max_connected_gr_amazon_movies_VAL, k=100)
#print(bc)


In [8]:
len(max_connected_gr_amazon_movies.edges('A1N0LCPR7O7OLL'))

4

In [9]:
#predict_amazon.df_summary_ratio_v.query("movie=='B000N2HD7I'").values[0].tolist()[1]
predict_amazon.df_summary_ratio_v

,movie,ratio_similar,review_count,review_count_old,review_count_new
0,630320497X,0.0,13,0,13
1,B0028O9VE2,0.0,197,0,197
2,B004BV9TPG,0.0,10,0,10
3,B00005A8TU,0.0,51,0,51
4,B004BR7XO4,0.0,1,0,1
...,...,...,...,...,...
350,080010255X,2.0,14,0,14
351,B004AAJJW6,0.0,1,0,1
352,B00006SKT1,0.0,3,0,3
353,B0000CDUXK,0.0,42,0,42


In [10]:
# this takes too much time to run, as well
# The eccentricity of a node v is the maximum distance from v to all other nodes in G.
# Returns: ecc – A dictionary of eccentricity values keyed by node.
# eccentricity(G, v=None, sp=None)
#e_model = nx.eccentricity(max_connected_gr_amazon_movies)
#e_test = nx.eccentricity(max_connected_gr_amazon_movies_VAL)

#print('Eccentricity of MODEL: {}\tTEST: {}'.format(e_model, e_test))

In [11]:
# below table shows the prediction for threshold>10% so don't pay attention to it
# also tot_rev_m_NEW & OLD will be double checked, seems like a bug
pd.options.display.max_rows = 1000
df_FINAL

,user,movie,prediction,reality,ngbr_ratio,tot_rev_m,tot_rev_m_OLD,tot_rev_m_NEW,n_old_edge,n_new_edge
0,AQEFHYQAPYCUL,B0006IWQIU,1,0,30.0,503,0,503,1,1
1,AQEFHYQAPYCUL,B000JGG6T4,1,0,16.0,297,0,297,1,1
2,AQEFHYQAPYCUL,6305176299,1,0,14.0,103,0,103,1,1
3,AQEFHYQAPYCUL,B000Z8GZYW,1,0,14.0,256,0,256,1,1
4,AQEFHYQAPYCUL,B00004CLDD,0,0,10.0,171,0,171,1,1
5,AQEFHYQAPYCUL,6301888901,0,0,10.0,71,0,71,1,1
6,AQEFHYQAPYCUL,B005ZMUP8K,0,0,10.0,596,0,596,1,1
7,AQEFHYQAPYCUL,B00006LLJ4,0,0,10.0,361,0,361,1,1
8,AQEFHYQAPYCUL,B003NTSBTC,0,0,8.0,367,0,367,1,1
9,AQEFHYQAPYCUL,B0001EYTK2,0,0,8.0,369,0,369,1,1


In [12]:
#pd.options.display.max_rows = 35500
df_FINAL_all.query("user=='A22GKUN35R09KN' and movie=='B000J6I0TS'")

,user,movie,prediction,reality,ngbr_ratio,tot_rev_m,tot_rev_m_OLD,tot_rev_m_NEW,n_old_edge,n_new_edge


In [16]:
'''
total_reviews = 0
        # total number of reviews that those 100 test users did in dataset-2
        for u in df_FINAL['user'].unique():
            n_movie_reviewed_new = df_FINAL.query("user == '" + u + "'")['n_new_edge'][0]
            total_reviews += n_movie_reviewed_new



for u in df_FINAL['user'].unique():
    print(u)
    print(df_FINAL.query("user == '" + u + "'")['n_new_edge'].values[0])
'''

'\ntotal_reviews = 0\n        # total number of reviews that those 100 test users did in dataset-2\n        for u in df_FINAL[\'user\'].unique():\n            n_movie_reviewed_new = df_FINAL.query("user == \'" + u + "\'")[\'n_new_edge\'][0]\n            total_reviews += n_movie_reviewed_new\n\n\n\nfor u in df_FINAL[\'user\'].unique():\n    print(u)\n    print(df_FINAL.query("user == \'" + u + "\'")[\'n_new_edge\'].values[0])\n'

In [14]:
len(df_FINAL['movie'].unique())

127

In [ ]:
predict_amazon.df_summary_ratio_v

In [ ]:
for t in thresholds:
    acc, tpr, fpr, tnr, fnr, ppv, f1 = predict_amazon.final_summary_threshold_based(df_FINAL, t)
    lst_t = [t,  acc, tpr, fpr, tnr, fnr, ppv, f1]
    lst_presicion_based.append(lst_t)

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

In [ ]:
'''
df_FINAL_copy = df_FINAL.copy()

threshold = 4
for i in range(len(df_FINAL_copy)):
    if df_FINAL_copy.loc[i, 'ngbr_ratio'] > threshold:
        df_FINAL_copy.loc[i, 'prediction'] = 1
    else:
        df_FINAL_copy.loc[i, 'prediction'] = 0
        
acc, tpr, fpr, tnr, ppv, f1 = predict_amazon.final_summary(df_FINAL_copy)

print(tpr)

#p_recall = tpr / (tpr + fnr)
#print(recall)
'''

In [ ]:
lst_true_val = df_FINAL.loc[:, 'reality'].values.tolist()
lst_ratio_val = df_FINAL.loc[:, 'ngbr_ratio'].values.tolist()

fpr, tpr, thresholds = metrics.roc_curve(lst_true_val, lst_ratio_val)  #, pos_label=2
lst_tpr = list(tpr); lst_fpr = list(fpr); lst_thresholds = list(thresholds) 

print(tpr)
print(thresholds)

In [ ]:


y_true = df_FINAL.loc[:, 'reality'].values.tolist()
y_pred = df_FINAL.loc[:, 'ngbr_ratio'].values.tolist()

roc_auc = roc_auc_score(y_true, y_pred)
print(roc_auc)

#precision = precision_score(y_true, y_pred, average='macro') # 'micro'  'weighted' None  average=None, zero_division=1
#print(precision)

precision_recall = precision_recall_curve(y_true, y_pred)
#print(list(precision_recall))

avg_precision = average_precision_score(y_true, y_pred)
#print(avg_precision)

#lst_tpr = 

#for i in range(len(lst_tpr)):
 #   lst_precision[i] = lst_tpr[i]/(lst_tpr[i] + lst_fpr[i])
    
#print(lst_precision)

In [ ]:
df_FINAL_copy = df_FINAL.copy()

threshold = 4
for i in range(len(df_FINAL_copy)):
    if df_FINAL_copy.loc[i, 'ngbr_ratio'] >= threshold:
        df_FINAL_copy.loc[i, 'prediction'] = 1
    else:
        df_FINAL_copy.loc[i, 'prediction'] = 0

tp = len(df_FINAL_copy.query("prediction==1 and reality==1").values)
fp = len(df_FINAL_copy.query("prediction==1 and reality==0").values)
tn = len(df_FINAL_copy.query("prediction==0 and reality==0").values)
fn = len(df_FINAL_copy.query("prediction==0 and reality==1").values)

p_precise = tp / (tp + fp)
p_recall = tp / (tp + fn)

print(p_precise)
print(p_recall)

In [ ]:
#u1=df_FINAL_copy.query("(prediction>0 or reality>0) and n_new_edge>0")
#u1=df_FINAL_copy.query("user=='AR15V2ULA2EYM' and (prediction>0 or reality>0) and n_new_edge>0 ").sort_values(by='ngbr_ratio', ascending=False)
#u1

In [ ]:
## NOTES
'''
Model1: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 80.23
Model2: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 86.89 .. 71.15 .. 
Model3: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 
Model4: 
n_movies = 40000 .. n_movies_v = 12000 .. n_test_users = 100 walk_steps = 40 .. top_neighbor = 50 >> AUC = 

'''